# Lab 2: Train Hugging Face Transformers on Amazon SageMaker

### Korean NLP Downstream task: Token Classification
---


## Introduction
---

바로 이전 모듈에서 기존에 온프레미스에서 개발했던 환경과 동일한 환경으로 모델을 빌드하고 훈련했습니다. 하지만 아래와 같은 상황들에서도 기존 환경을 사용하는 것이 바람직할까요?

- 온프레미스의 GPU는 총 1장으로 훈련 시간이 너무 오래 소요됨
- 가용 서버 대수가 2대인데 10개의 딥러닝 모델을 동시에 훈련해야 함
- 필요한 상황에만 GPU를 활용
- 기타 등등

Amazon SageMaker는 데이터 과학자들 및 머신 러닝 엔지니어들을 위한 완전 관리형 머신 러닝 서비스로 훈련 및 추론 수행 시 인프라 설정에 대한 추가 작업이 필요하지 않기에, 단일 GPU 기반의 딥러닝 훈련을 포함한 멀티 GPU 및 멀티 인스턴스 분산 훈련을 보다 쉽고 빠르게 수행할 수 있습니다. SageMaker는 다양한 유즈케이스들에 적합한 예제들을 지속적으로 업데이트하고 있으며, 한국어 세션 및 자료들도 제공되고 있습니다.

### Notes

이미 기본적인 Hugging Face 용법 및 자연어 처리에 익숙하신 분들은 앞 모듈을 생략하고 이 모듈부터 핸즈온을 시작하셔도 됩니다.
이 노트북은 SageMaker 기본 API를 참조하므로, SageMaker Studio, SageMaker 노트북 인스턴스 또는 AWS CLI가 설정된 로컬 시스템에서 실행해야 합니다. SageMaker Studio 또는 SageMaker 노트북 인스턴스를 사용하는 경우 PyTorch 기반 커널을 선택하세요.
훈련 job 수행 시 최소 `ml.g4dn.xlarge` 이상의 훈련 인스턴스가 필요하며, `ml.p3.8xlarge`나 `ml.p3.16xlarge` 인스턴스를 권장합니다. 만약 인스턴스 사용에 제한이 걸려 있다면 Request a service quota increase for SageMaker resources를 참조하여 인스턴스 제한을 해제해 주세요.

### References
- Hugging Face Tutorial: https://huggingface.co/docs/transformers/training
- 네이버, 창원대가 함께하는 NLP Challenge GitHub: https://github.com/naver/nlp-challenge
- 네이버, 창원대가 함께하는 NLP Challenge 리더보드 및 라이센스: http://air.changwon.ac.kr/?page_id=10


## 1. Setup Environments
---

### Import modules

In [1]:
import boto3
import sagemaker
import sagemaker.huggingface

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20220317T150353
sagemaker bucket: sagemaker-us-east-1-143656149352
sagemaker session region: us-east-1


In [2]:
import os
import sys
import logging
import argparse
import torch
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertTokenizerFast, BertConfig, BertForTokenClassification

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

### Load Feature set

In [3]:
train_dir = 'ner_train'
valid_dir = 'ner_valid'
train_dict = torch.load(os.path.join(train_dir, 'train_features.pt'))
valid_dict = torch.load(os.path.join(valid_dir, 'valid_features.pt'))

train_ids, train_attention_masks, train_labels = train_dict['input_ids'], train_dict['attention_mask'], train_dict['labels']     
valid_ids, valid_attention_masks, valid_labels = valid_dict['input_ids'], valid_dict['attention_mask'], valid_dict['labels']   
        
logger.info(f"Loaded train_dataset length is: {len(train_dict['input_ids'])}")
logger.info(f"Loaded test_dataset length is: {len(valid_dict['input_ids'])}")

[{2817287866.py:9} INFO - Loaded train_dataset length is: 52416
[{2817287866.py:10} INFO - Loaded test_dataset length is: 13104


In [4]:
# num_debug_samples = 500
# train_ids = train_ids[:num_debug_samples, :]
# train_attention_masks = train_attention_masks[:num_debug_samples, :]
# train_labels = train_labels[:num_debug_samples, :]

# valid_ids = valid_ids[:num_debug_samples, :]
# valid_attention_masks = valid_attention_masks[:num_debug_samples, :]
# valid_labels = valid_labels[:num_debug_samples, :]

In [5]:
# train_dict = {'input_ids': train_ids, 'attention_mask': train_attention_masks, 'labels': train_labels}
# valid_dict = {'input_ids': valid_ids, 'attention_mask': valid_attention_masks, 'labels': valid_labels}

# torch.save(train_dict, os.path.join(train_dir, 'train_features.pt'))
# torch.save(valid_dict, os.path.join(valid_dir, 'valid_features.pt'))

### Uploading data to Amazon S3 Bucket

SageMaker 훈련을 위해 데이터셋을 S3로 업로드합니다.

In [6]:
bucket = sess.default_bucket()

# s3 key prefix for the data
s3_prefix = 'samples/datasets/naver-changwon-ner'

# save train_dataset to s3
train_input_path = f's3://{bucket}/{s3_prefix}/train'
valid_input_path = f's3://{bucket}/{s3_prefix}/valid'

In [7]:
%%time
!aws s3 cp {train_dir} {train_input_path} --recursive --quiet
!aws s3 cp {valid_dir} {valid_input_path} --recursive --quiet

CPU times: user 31.4 ms, sys: 55 ms, total: 86.4 ms
Wall time: 3.17 s


<br>

## 2. Training with Native Hugging Face (PyTorch Framework)

---

### Overview and Training Script

SageMaker에 대한 대표적인 오해가 여전히 많은 분들이 SageMaker 훈련을 위해 소스 코드를 전면적으로 수정해야 한다고 생각합니다. 하지만, 실제로는 별도의 소스 코드 수정 없이 기존 여러분이 사용했던 파이썬 스크립트에 SageMaker 훈련에 필요한 SageMaker 전용 환경 변수들만 추가하면 됩니다. 

SageMaker 훈련은 훈련 작업을 호출할 때, 1) 훈련 EC2 인스턴스 프로비저닝 - 2) 컨테이너 구동을 위한 도커 이미지 및 훈련 데이터 다운로드 - 3) 컨테이너 구동 - 4) 컨테이너 환경에서 훈련 수행 - 5) 컨테이너 환경에서 S3의 특정 버킷에 저장 - 6) 훈련 인스턴스 종료로 구성됩니다. 따라서, 훈련 수행 로직은 아래 예시와 같이 기존 개발 환경과 동일합니다.

```python
/opt/conda/bin/python train.py --epochs 5 --train_batch_size 32 ...
```

이 과정에서 컨테이너 환경에 필요한 환경 변수(예: 모델 경로, 훈련 데이터 경로) 들은 사전에 지정되어 있으며, 이 환경 변수들이 설정되어 있어야 훈련에 필요한 파일들의 경로를 인식할 수 있습니다. 대표적인 환경 변수들에 대한 자세한 내용은 https://github.com/aws/sagemaker-containers#important-environment-variables 을 참조하세요.

In [8]:
from sagemaker.huggingface import HuggingFace
import time
instance_type = 'ml.p3.8xlarge'
num_gpus = 4
instance_count = 1
batch_size_native = 32
learning_rate_native = float('5e-5')
batch_size = batch_size_native
model_id = 'bert-base-multilingual-cased' 
tokenizer_id = model_id
learning_rate = learning_rate_native / batch_size_native * batch_size * num_gpus * instance_count
logger.info(f'learning_rate: {learning_rate}')

# hyperparameters, which are passed into the training job
hyperparameters = {
    'n_gpus': num_gpus,                 # number of GPUs per instance
    'epochs': 3,                        # number of training epochs
    'seed': 42,                         # random seed
    'train_batch_size': batch_size,     # batch size for training
    'eval_batch_size': batch_size*2,    # batch size for evaluation
    'learning_rate': learning_rate,     # learning rate used during training
    'tokenizer_id': tokenizer_id,       # pre-trained tokenizer
    'model_id': model_id                # pre-trained model
}

[{1923326789.py:12} INFO - learning_rate: 0.0002


In [9]:
# define Training Job Name 
job_name = f'kornlp-ner-training-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
chkpt_s3_path = f's3://{bucket}/{s3_prefix}/native/checkpoints'

# create the Estimator
sm_estimator = HuggingFace(
    entry_point           = 'train.py',         # fine-tuning script used in training jon
    source_dir            = './scripts',        # directory where fine-tuning script is stored
    instance_type         = instance_type,      # instances type used for the training job
    instance_count        = instance_count,     # the number of instances used for training
    base_job_name         = job_name,           # the name of the training job
    role                  = role,               # IAM role used in training job to access AWS ressources, e.g. S3
    transformers_version  = '4.17.0',           # the transformers version used in the training job
    pytorch_version       = '1.10.2',           # the pytorch_version version used in the training job
    py_version            = 'py38',             # the python version used in the training job
    hyperparameters       = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler      = True,
    debugger_hook_config  = False, 
    checkpoint_s3_uri     = chkpt_s3_path,
    checkpoint_local_path ='/opt/ml/checkpoints',  
)

logger.info(f'training job name: {job_name}')

[{666484959.py:23} INFO - training job name: kornlp-ner-training-2022-06-13-05-07-10


`fit()` 메소드를 호출하여 훈련 job을 시작합니다. `fit()` 메소드의 인자값 중 `wait=True`로 설정할 경우에는 동기(synchronous) 방식으로 동직하게 되며, `wait=False`일 경우에는 비동기(aynchronous) 방식으로 동작하여 여러 개의 훈련 job을 동시에 실행할 수 있습니다.

In [10]:
# define a data input dictonary with our uploaded s3 uris
data = {
    'train': train_input_path,
    'valid': valid_input_path
}

# starting the train job with our uploaded datasets as input
sm_estimator.fit(data, wait=False)
train_job_name = sm_estimator.latest_training_job.job_name

[{session.py:588} INFO - Creating training-job with name: kornlp-ner-training-2022-06-13-05-07-10-2022-06-13-05-07-24-970


### View Training Job
SageMaker 콘솔 창에서 훈련 내역을 직접 확인할 수도 있지만, 아래 코드 셀에서 생성되는 링크를 클릭하면 더 편리하게 훈련 내역을 확인할 수 있습니다.

In [11]:
from IPython.core.display import display, HTML

def make_console_link(region, train_job_name, train_task='[Training]'):
    train_job_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{train_job_name}">Training Job</a></b>'   
    cloudwatch_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={train_job_name};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'
    return train_job_link, cloudwatch_link  
        
train_job_link, cloudwatch_link = make_console_link(region, train_job_name, '[Hugging Face Training - Native]')

display(HTML(train_job_link))
display(HTML(cloudwatch_link))

### Wait for the training jobs to complete
훈련이 완료될 때까지 기다립니다. `estimator.fit(...)`에서 `wait=False`로 설정한 경우, 아래 코드 셀의 주석을 해제 후 실행하여 동기 방식으로 변경할 수도 있습니다. 훈련 완료까지는 약 15-20분의 시간이 소요됩니다.

In [ ]:
sess.logs_for_job(job_name=train_job_name, wait=True)

2022-06-13 05:08:19 Starting - Preparing the instances for training.........
2022-06-13 05:09:45 Downloading - Downloading input data......
2022-06-13 05:10:56 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2022-06-13 05:14:50,602 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-13 05:14:50,641 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-13 05:14:50,648 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-13 05:14:51,225 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
2022-06-1


2022-06-13 05:14:47 Training - Training image download completed. Training in progress.[{train.py:128} INFO - ***** Arguments *****
[{train.py:129} INFO - epochs=3
seed=42
train_batch_size=32
eval_batch_size=64
warmup_steps=100
learning_rate=0.0002
disable_tqdm=False
debug=False
tokenizer_id=bert-base-multilingual-cased
model_id=bert-base-multilingual-cased
output_data_dir=/opt/ml/output/data
model_dir=/opt/ml/model
n_gpus=4
train_dir=/opt/ml/input/data/train
valid_dir=/opt/ml/input/data/valid
chkpt_dir=/opt/ml/checkpoints
[{train.py:165} INFO - Loaded train_dataset length is: 52416
[{train.py:166} INFO - Loaded test_dataset length is: 13104
Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 38.9kB/s]
Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]
Downloading: 100%|██████████| 972k/972k [00:00<00:00, 38.8MB/s]
Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]
Downloading: 100%|██████████| 1.87M/1.87M [00:00<00:

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Model weights saved in /opt/ml/model/pytorch_model.bin
Model weights saved in /opt/ml/model/pytorch_model.bin
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
100%|██████████| 52/52 [00:11<00:00,  4.61it/s]
2022-06-13 05:15:56,517 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2022-06-13 05:16:03 Uploading - Uploading generated training model
2022-06-13 05:17:38 Completed - Training job completed


### Copy model artifacts from S3 to local path

훈련된 모델 파라메터는 `model.tar.gz`로 압축되어 S3에 저장됩니다. 만약 SageMaker 상에서 훈련한 모델을 곧바로 배포한다면, 아래 코드 셀을 실행할 필요는 없지만, 로컬/개발 환경에서 훈련된 모델을 간단히 테스트하거나 다른 환경에서 모델을 서빙할 때는 S3에 저장된 모델을 다운로드하셔야 합니다.

In [15]:
local_model_dir = './model'
!rm -rf {local_model_dir}
s3_model_path = sm_estimator.model_data
os.makedirs(local_model_dir, exist_ok=True)

In [16]:
%%bash -s "$local_model_dir" "$s3_model_path"
aws s3 cp $2 $1
cd $1
tar -xzvf model.tar.gz

download: s3://sagemaker-us-east-1-143656149352/kornlp-ner-training-2022-06-13-05-07-10-2022-06-13-05-07-24-970/output/model.tar.gz to model/model.tar.gz
id2tag.json
config.json
tokenizer.json
training_args.bin
tag2id.json
special_tokens_map.json
vocab.txt
pytorch_model.bin
tag2entity.json
tokenizer_config.json


In [17]:
%store s3_model_path local_model_dir model_id tokenizer_id

Stored 's3_model_path' (str)
Stored 'local_model_dir' (str)
Stored 'model_id' (str)
Stored 'tokenizer_id' (str)


<br>

## 3. Prediction
---
### Load fine-tuned model

In [18]:
import json
with open(os.path.join(local_model_dir, 'tag2id.json'), 'r') as f:
    tag2id = json.loads(f.read())

with open(os.path.join(local_model_dir, 'id2tag.json'), 'r') as f:
    id2tag = json.loads(f.read())    

with open(os.path.join(local_model_dir, 'tag2entity.json'), 'r') as f:
    tag2entity = json.loads(f.read())

tag2id = {k:int(v) for k,v in tag2id.items()}     
id2tag = {int(k):v for k,v in id2tag.items()}    

In [19]:
# from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
# tokenizer = BertTokenizerFast.from_pretrained(tokenizer_id)
# tokenizer.save(f'{local_model_path}/tokenizer.json')

In [20]:
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
tokenizer = BertTokenizerFast.from_pretrained(f'{local_model_dir}')
model = BertForTokenClassification.from_pretrained(model_id, num_labels=len(id2tag))
model.config.id2label = id2tag
model.config.label2id = tag2id
model.load_state_dict(torch.load(f'{local_model_dir}/pytorch_model.bin'))
model = model.eval()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [21]:
def predict_fn_token(example, model):
 
    inputs_with_offsets = tokenizer(example, return_offsets_mapping=True, return_tensors='pt')
    inputs = inputs_with_offsets.copy()
    inputs.pop('offset_mapping')
    tokens = inputs_with_offsets.tokens()
    offsets = inputs_with_offsets["offset_mapping"]
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = inputs.to(device)
    model = model.to(device)
    
    output = model(**inputs)
    softmax_fn = nn.Softmax(dim=-1)
    softmax_output = softmax_fn(output['logits'])
    prob, pred = torch.max(softmax_output, dim=-1)
    pred_str_lst = [model.config.id2label[id] for id in pred.squeeze().tolist()]
    prob_lst, pred_lst = prob.squeeze().tolist(), pred.squeeze().tolist()
    offsets_lst = offsets.squeeze().tolist()
    
    df = pd.DataFrame(zip(tokens, pred_str_lst, prob_lst, pred_lst, offsets_lst), 
                  columns=['token', 'tag', 'score', 'label', 'offset'])
    
    return df


def predict_fn_word(example, model):

    from transformers import pipeline
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device_id = -1 if device.type == "cpu" else 0
    
    nlp = pipeline("ner", model=model.to(device), device=device_id, 
                   tokenizer=tokenizer, aggregation_strategy='average')
    results = nlp(example)
    
    entity_lst, score_lst, word_lst, start_lst, end_lst = [], [], [], [], []
    tag2entity[''] = '-'

    for result in results:
        entity = tag2entity[result['entity_group']]
        score = result['score']
        word = result['word']
        start = result['start']
        end = result['end']

        entity_lst.append(entity)
        score_lst.append(score)
        word_lst.append(word)
        start_lst.append(start)
        end_lst.append(end)

    df = pd.DataFrame(zip(word_lst, entity_lst, score_lst, start_lst, end_lst), 
                      columns=['word', 'entity', 'score', 'start', 'end'])
    return df

### Example
여러분만의 샘플 문장을 만들어서 자유롭게 추론을 수행해 보세요.

In [22]:
example = '아마존 SageMaker는 머신 러닝 통합 엔드투엔드 관리형 서비스로 2017년 re:Invent 행사가 열린 라스베가스에서 발표되었다.'
token_df = predict_fn_token(example, model)
token_df.head(20)

,token,tag,score,label,offset
0,[CLS],-,0.999695,2,"[0, 0]"
1,아,ORG_B,0.971075,3,"[0, 1]"
2,##마,ORG_B,0.971737,3,"[1, 2]"
3,##존,ORG_B,0.969591,3,"[2, 3]"
4,Sage,TRM_B,0.557998,8,"[4, 8]"
5,##M,TRM_B,0.546546,8,"[8, 9]"
6,##aker,TRM_B,0.550420,8,"[9, 13]"
7,##는,TRM_B,0.540432,8,"[13, 14]"
8,머신,TRM_B,0.943289,8,"[15, 17]"
9,러,TRM_I,0.683402,9,"[18, 19]"


In [23]:
from transformers import pipeline
nlp = pipeline("ner", model=model.to('cpu'), device=-1, 
               tokenizer=tokenizer, aggregation_strategy='average')
results = nlp(example)

In [24]:
word_df = predict_fn_word(example, model)
word_df

,word,entity,score,start,end
0,아마존,Organization,0.970801,0,3
1,SageMaker는 머신,Term,0.746069,4,17
2,러닝,Term,0.695091,18,20
3,통합,-,0.534267,21,23
4,엔드투엔드,Term,0.540918,24,29
5,관리형,-,0.532034,30,33
6,서비스로,Term,0.483128,34,38
7,2017년,Date,0.996233,39,44
8,re :,Event,0.802971,45,48
9,Invent 행사가,Event,0.731568,48,58
